In [18]:
import os
import torch
import json
import csv
import argparse
import numpy as np
from PIL import Image
import torch.nn as nn
import multiprocessing
from model import ResNet18
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.models as models
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from config import args as default_args
from utils import DotDict, parse_arguments
import torchvision.transforms as transforms
from dataloader import CocoClassDatasetRandom

In [19]:
from config import args

In [20]:
def change_to_3_channel(x):
    if x.size()[0] == 1:
        return x.repeat(3, 1, 1)
    return x

train_transform = transforms.Compose([
                                    # transforms.Resize(size=(224,224)),
                                    # transforms.RandomCrop(224, padding=4),
                                    # transforms.RandomHorizontalFlip(), 
                                    transforms.ToTensor(),
                                    change_to_3_channel,
                                    transforms.Normalize(args.mean, args.std)
                                ])

val_transform = transforms.Compose([
    # transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    change_to_3_channel,
    transforms.Normalize(args.mean, args.std),
])

train_dataset = CocoClassDatasetRandom(images_path = args.train_images_path, annotation_path = args.train_annotation_path, transform = train_transform)
val_dataset = CocoClassDatasetRandom(images_path = args.val_images_path, annotation_path = args.val_annotation_path, transform = val_transform)

print(f'----> number of workers: {args.num_workers}')

loading annotations into memory...
Done (t=14.42s)
creating index...
index created!
loading annotations into memory...
Done (t=4.48s)
creating index...
index created!
----> number of workers: 40


In [21]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

In [22]:
net = ResNet18() 
inp_features = net.linear.in_features
net.linear = torch.nn.Linear(in_features=inp_features, out_features=args.num_classes)

In [23]:
batch_x, batch_y = next(iter(trainloader))

print('-----> verify if model is run on random data')

-----> verify if model is run on random data


In [24]:
batch_x.shape, batch_y.shape

(torch.Size([4, 3, 224, 224]), torch.Size([4, 1, 40]))

In [25]:
outputs = net(batch_x)

print(f'Outputs shape: {outputs.shape}, batch labels shape: {batch_y.shape}') 

Outputs shape: torch.Size([4, 40]), batch labels shape: torch.Size([4, 1, 40])


In [26]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr = args.base_learning_rate, momentum=args.momentum, weight_decay=args.weight_decay)

In [27]:
print(f'Initial random loss: {criterion(outputs, batch_y.squeeze().float())}')

Initial random loss: 0.7418841123580933


In [29]:
batch_y.squeeze()

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
predicted = torch.round(torch.sigmoid(outputs))
predicted

tensor([[0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
         0., 0., 0., 0.],
        [0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
         0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
         1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
         0., 0., 0., 0.],
        [0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
         0., 0., 0., 0.]], grad_fn=<RoundBackward0>)

In [28]:
predicted.eq(batch_y.squeeze().float())

tensor([[False,  True,  True, False, False,  True, False,  True,  True, False,
         False,  True, False, False,  True, False,  True,  True,  True, False,
         False, False,  True, False, False, False,  True,  True, False,  True,
         False, False, False,  True,  True, False,  True,  True,  True,  True],
        [False,  True, False, False, False, False, False,  True,  True, False,
          True,  True, False, False,  True, False,  True,  True,  True,  True,
         False, False,  True, False, False, False,  True,  True, False,  True,
         False,  True, False,  True,  True, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True, False,  True,  True,  True,
         False,  True, False, False,  True, False,  True, False, False,  True,
         False, False,  True, False, False, False,  True, False, False,  True,
         False,  True, False, False,  True, False,  True,  True,  True,  True],
        [False, False,  True, False, False,  True

In [30]:
correct = predicted.eq(batch_y.squeeze().float()).cpu().sum()
correct

tensor(80)

In [36]:
random_acc = correct/(batch_size * args.num_classes) * 100
print(f'Initial random acc: {random_acc} %')

Initial random acc: 50.0 %


In [38]:
batch_size * args.num_classes

160